In [29]:
import pandas as pd
from Bio import SeqIO
from collections import Counter
import numpy as np

## 1. Организм Kluyveromyces lactis strain GG799 chromosome D

##### Самописная функция для вычисления качества выравнивания по суммарной энтропии

In [42]:
def score_align(list_align):
    entropy, nucl_counter, min_len, n_align = 0, Counter(), min(map(lambda x: len(x), list_align)), len(list_align)
    for index_nucl in range(min_len):
        tempory_nucl = []
        for align in list_align:
            tempory_nucl.append(align[index_nucl])
        nucl_counter = Counter(tempory_nucl)
        for nucl in ['A', 'C', 'G', 'T', '-']:
            p = nucl_counter[nucl] / n_align
            if p != 0:
                entropy += -p * np.log2(p)
    return entropy

### Подсчёт энтропии для ДНК последовательностей

In [52]:
align_1, align_2, align_3, align_4 = [], [], [], []
for record_1, record_2, record_3, record_4 in zip(SeqIO.parse("data/clustal_omega_aln.fasta.txt", "fasta"),
                                       SeqIO.parse("data/mafft.txt", "fasta"), 
                                       SeqIO.parse("data/muscle_aln.fasta.txt", "fasta"),
                                       SeqIO.parse("data/prank_aln.txt", "fasta")):
    align_1.append(record_1.seq)
    align_2.append(record_2.seq)
    align_3.append(record_3.seq)
    align_4.append(record_4.seq)

In [53]:
align_test = pd.DataFrame({'time/sec' : [8, 7, 5, 'nan'],
              'aligner' : ['clustalomega', 'muscle', 'mafft', 'prank'],
              'Aligment entropy' : [score_align(align_1), score_align(align_2), score_align(align_3), score_align(align_4)],
              'params' : ['--infile clustalo-I20200318-195558-0152-20387981-p2m.upfile --threads 8 --MAC-RAM 8000 --verbose --guidetree-out clustalo-I20200318-195558-0152-20387981-p2m.dnd --outfmt fa --outfile clustalo-I20200318-195558-0152-20387981-p2m.fasta --output-order tree-order --seqtype dna', '-in muscle-I20200318-200022-0566-71885333-p2m.upfile -verbose -log muscle-I20200318-200022-0566-71885333-p2m.output -quiet -fasta -out muscle-I20200318-200022-0566-71885333-p2m.fasta', '--thread 8 --anysymbol --bl 62 --op 1.53 --ep 0.123 --reorder --retree 2 --treeout --maxiterate 2 --nuc mafft-I20200318-200032-0321-28117674-p2m.upfile', 'default']
})

In [54]:
align_test.sort_values('Aligment entropy')

,time/sec,aligner,Aligment entropy,params
2,5,mafft,992.614875,--thread 8 --anysymbol --bl 62 --op 1.53 --ep ...
0,8,clustalomega,996.710397,--infile clustalo-I20200318-195558-0152-203879...
1,7,muscle,1015.653870,-in muscle-I20200318-200022-0566-71885333-p2m....
3,nan,prank,1113.487577,default


### Подсчёт энтропии для аминокислотных последовательностей

In [60]:
align_1, align_2, align_3 = [], [], []
for record_1, record_2, record_3 in zip(SeqIO.parse("data/clustalw_protein_aln.txt", "fasta"),
                                       SeqIO.parse("data/mafft_protein_aln.txt", "fasta"), 
                                       SeqIO.parse("data/tcoffee_protein_aln.txt", "fasta")):

    align_1.append(record_1.seq)
    align_2.append(record_2.seq)
    align_3.append(record_3.seq)

In [61]:
align_test = pd.DataFrame({'time/sec' : [2, 4, 1],
              'aligner' : ['clustalomega', 'muscle', 't-coffee'],
              'Aligment entropy' : [score_align(align_1), score_align(align_2), score_align(align_3)],
              'params' : ['--infile clustalo-I20200318-212004-0854-15550679-p2m.upfile --threads 8 --MAC-RAM 8000 --verbose --guidetree-out clustalo-I20200318-212004-0854-15550679-p2m.dnd --outfmt fa --outfile clustalo-I20200318-212004-0854-15550679-p2m.fasta --output-order tree-order --seqtype protein',
                         '--thread 8 --anysymbol --bl 62 --op 1.53 --ep 0.123 --reorder --retree 2 --treeout --maxiterate 2 mafft-I20200318-212013-0043-83854256-p2m.upfile',
                         '-in tcoffee-I20200318-213627-0298-7550595-p2m.upfile -case=upper -n_core=8 -output=clustalw,msf,phylip,score_html,fasta -outorder=aligned -type=protein; echo']
})

In [62]:
align_test.sort_values('Aligment entropy')

,time/sec,aligner,Aligment entropy,params
1,4,muscle,98.043261,--thread 8 --anysymbol --bl 62 --op 1.53 --ep ...
0,2,clustalomega,108.452171,--infile clustalo-I20200318-212004-0854-155506...
2,1,t-coffee,360.041359,-in tcoffee-I20200318-213627-0298-7550595-p2m....


### Для strange_aln как мне кажется, проблема в том, что там много ситуаций вида выравнивания:
#### AGCT
#### - - - -
#### - - - -
#### - - - -
#### GCCT
### т.е. нужно как-то инвертировать гэпы